In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/ml-models'


In [3]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [4]:
import os
import pandas as pd
from mlrun.datastore import DataItem
from mlrun.artifacts import get_model
from cloudpickle import load
from mlrun.mlutils import eval_class_model

def cox_test(
    context,
    models_path: DataItem, 
    test_set: DataItem,
    label_column: str,
    plots_dest: str = "plots",
    model_evaluator = None
) -> None:
    """Test one or more classifier models against held-out dataset
    
    Using held-out test features, evaluates the peformance of the estimated model
    
    Can be part of a kubeflow pipeline as a test step that is run post EDA and 
    training/validation cycles
    
    :param context:         the function context
    :param model_file:      model artifact to be tested
    :param test_set:        test features and labels
    :param label_column:    column name for ground truth labels
    :param score_method:    for multiclass classification
    :param plots_dest:      dir for test plots
    :param model_evaluator: WIP: specific method to generate eval, passed in as string
                            or available in this folder
    """  
    xtest = test_set.as_df()
    ytest = xtest.pop(label_column)
    
    model_file, model_obj, _ = get_model(models_path.url, suffix='.pkl')
    model_obj = load(open(str(model_file), "rb"))

    try:
        # there could be different eval_models, type of model (xgboost, tfv1, tfv2...)
        if not model_evaluator:
            # binary and multiclass
            eval_metrics = eval_class_model(context, xtest, ytest, model_obj)

        # just do this inside log_model?
        model_plots = eval_metrics.pop("plots")
        model_tables = eval_metrics.pop("tables")
        for plot in model_plots:
            context.log_artifact(plot, local_path=f"{plots_dest}/{plot.key}.html")
        for tbl in model_tables:
            context.log_artifact(tbl, local_path=f"{plots_dest}/{plot.key}.csv")

        context.log_results(eval_metrics)
    except:
        #dummy log:
        context.log_dataset("cox-test-summary", df=model_obj.summary, index=True, format="csv")
        context.logger.info("cox tester not implemented")

In [5]:
# nuclio: end-code

### mlconfig

In [6]:
from mlrun import mlconf
import os

mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [7]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("cox_test")

# add metadata (for templates and reuse)
fn.spec.default_handler = "cox_test"
fn.spec.description = "test a classifier using held-out or new data"
fn.metadata.categories = ["ml", "test"]
fn.metadata.labels = {"author": "yjb", "framework": "survival"}
fn.export("function.yaml")

[mlrun] 2020-05-14 22:35:36,274 function spec saved to path: function.yaml


In [8]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc('nfsvol', 'nfsvol', '/home/jovyan/data'))

In [9]:
task_params = {
    "name" : "tasks cox test",
    "params": {
        "label_column"  : "labels",
        "plots_dest"    : "churn/test/plots"}}

In [10]:
DATA_URL = "https://raw.githubusercontent.com/yjb-ds/testdata/master/data/churn-tests.csv"

In [11]:
from mlrun import run_local, NewTask

run = run_local(NewTask(**task_params),
                handler=cox_test,
                inputs={"test_set": DATA_URL,
                        "models_path"   : "models/cox"},
               workdir=mlconf.artifact_path+"/churn")

[mlrun] 2020-05-14 22:35:39,440 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 22:35:39,440 starting run tasks cox test uid=b1cb1fbca41b49028a0d10d02e6b967a  -> http://mlrun-api:8080
[mlrun] 2020-05-14 22:35:39,474 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 22:35:40,081 log artifact cox-test-summary at /User/artifacts/cox-test-summary.csv, size: 3429, db: Y
[mlrun] 2020-05-14 22:35:40,082 cox tester not implemented



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...2e6b967a,0,May 14 22:35:39,completed,tasks cox test,v3io_user=adminkind=handlerowner=adminhost=jupyter-f597579bb-l8hbz,test_setmodels_path,label_column=labelsplots_dest=churn/test/plots,,cox-test-summary


to track results use .show() or .logs() or in CLI: 
!mlrun get run b1cb1fbca41b49028a0d10d02e6b967a --project default , !mlrun logs b1cb1fbca41b49028a0d10d02e6b967a --project default
[mlrun] 2020-05-14 22:35:40,143 run executed, status=completed


In [12]:
run = fn.run(
    NewTask(**task_params),
    inputs={
        "test_set": DATA_URL,
        "models_path"   : "models/cox"},
    workdir=os.path.join(mlconf.artifact_path, "churn"))

[mlrun] 2020-05-14 22:35:41,260 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 22:35:41,260 starting run tasks cox test uid=cf114840236a441391e8a42baad5613f  -> http://mlrun-api:8080
[mlrun] 2020-05-14 22:35:41,349 Job is running in the background, pod: tasks-cox-test-mbwsr
[mlrun] 2020-05-14 22:35:44,844 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 22:35:45,606 warning!, server (0.4.7) and client (0.4.8) ver dont match
[mlrun] 2020-05-14 22:35:45,790 log artifact cox-test-summary at /User/artifacts/cox-test-summary.csv, size: 3429, db: Y
[mlrun] 2020-05-14 22:35:45,790 cox tester not implemented

[mlrun] 2020-05-14 22:35:45,818 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...aad5613f,0,May 14 22:35:45,completed,tasks cox test,host=tasks-cox-test-mbwsrkind=jobowner=adminv3io_user=admin,models_pathtest_set,label_column=labelsplots_dest=churn/test/plots,,cox-test-summary


to track results use .show() or .logs() or in CLI: 
!mlrun get run cf114840236a441391e8a42baad5613f  , !mlrun logs cf114840236a441391e8a42baad5613f 
[mlrun] 2020-05-14 22:35:47,550 run executed, status=completed
